In [1]:
import pandas as pd
import numpy as np
import sqlite3
import time

How many records does each dataset have in the sqlite db? 

In [2]:
query = """
SELECT COUNT(*) AS hopteaming_count
FROM hopteaming
"""

with sqlite3.connect('../data/hop_teaming.sqlite') as db:
    hopteaming_count = pd.read_sql(query, db)

hopteaming_count.head(3)

,hopteaming_count
0,34184634


In [3]:
query = """
SELECT COUNT(*) AS taxonomy_count
FROM taxonomy
"""

with sqlite3.connect('../data/hop_teaming.sqlite') as db:
    taxonomy_count = pd.read_sql(query, db)

taxonomy_count.head(3)

,taxonomy_count
0,874


In [4]:
query = """
SELECT COUNT(*) AS npi_count
FROM npi
"""

with sqlite3.connect('../data/hop_teaming.sqlite') as db:
    npi_count = pd.read_sql(query, db)

npi_count.head(3)

,npi_count
0,8180697


In [5]:
query = """
SELECT COUNT(*) AS zip_count
FROM zip_cbsa
"""

with sqlite3.connect('../data/hop_teaming.sqlite') as db:
    zip_count = pd.read_sql(query, db)

zip_count.head(3)

,zip_count
0,47598


In [6]:
query = """
SELECT * 
FROM zip_cbsa
WHERE cbsa = 34980
"""
start_time = time.time()

with sqlite3.connect('../data/data2/hop_teaming.sqlite') as db:
    nashville_cbsa = pd.read_sql(query, db)
    
end_time = time.time()
print("Execution time:", end_time - start_time, "seconds")

nashville_cbsa.head(3)

Execution time: 0.04092121124267578 seconds


,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,37010,34980,ADAMS,TN,0.358611,0.909091,1.0,0.370867
1,37011,34980,ANTIOCH,TN,1.000000,1.000000,1.0,1.000000
2,37012,34980,ALEXANDRIA,TN,0.219592,0.043478,0.0,0.211719


After creating the sqlite database for the four datasets (hopteaming, taxonomy, zip_cbsa, and npi), merge datasets. 
- Next, download the **taxonomy** code to classification crosswalk from the National Uniform Claim Committee (https://www.nucc.org/index.php/code-sets-mainmenu-41/provider-taxonomy-mainmenu-40/csv-mainmenu-57). Using the primary taxonomy code, match each provider to a classification (from the Classification column).
- Finally, you need the **Zip code** to CBSA crosswalk from here: https://www.huduser.gov/portal/datasets/usps_crosswalk.html. Match each provider to a CBSA using the Business Zip code. Note that the zipcodes in the nppes dataset are either 5 or 9 digits, and be mindful that leading zeros might be dropped when reading the dataset into a dataframe. This can be used if you want to filter to providers just in the Nashville CBSA.

In [7]:
query = """
SELECT *
FROM npi 
LEFT JOIN taxonomy
ON npi.healthcare_provider_taxonomy_code = taxonomy.code
INNER JOIN zip_cbsa
ON npi.zip = zip_cbsa.zip
WHERE cbsa = 34980;
"""
start_time = time.time()

with sqlite3.connect('../data/hop_teaming.sqlite') as db:
    npi_nash  = pd.read_sql(query, db)
    
end_time = time.time()
print("Execution time:", end_time - start_time, "seconds")
    
npi_nash.head(3)

Execution time: 9.297683954238892 seconds


,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,...,display_name,section,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,1013983808,1.0,None,SMITH,BEVERLY,KAY,MRS.,None,NP,836 DIXIE BEE RD,...,Family Nurse Practitioner,Individual,37010,34980,ADAMS,TN,0.358611,0.909091,1.0,0.370867
1,1346216199,1.0,None,GERTONSON,STEVEN,FREDRICK,MR.,None,CRNA,1950 BRADBURY RD,...,Certified Registered Nurse Anesthetist,Individual,37010,34980,ADAMS,TN,0.358611,0.909091,1.0,0.370867
2,1649660085,1.0,None,SMITH,KALEENA,DANIELLE,None,None,"M.S., CCC-SLP",948 NECTAR CT,...,Speech-Language Pathologist,Individual,37010,34980,ADAMS,TN,0.358611,0.909091,1.0,0.370867


In [8]:
npi_nash.index

RangeIndex(start=0, stop=50824, step=1)

In [ ]:
# query = """
# SELECT *
# FROM npi 
# LEFT JOIN taxonomy
# ON npi.healthcare_provider_taxonomy_code = taxonomy.code
# INNER JOIN zip_cbsa
# ON npi.zip = zip_cbsa.zip
# WHERE cbsa = 34980;
# """
# start_time = time.time()

# with sqlite3.connect('../data/hop_teaming.sqlite') as db:
#     npi_nash  = pd.read_sql(query, db)
    
# end_time = time.time()
# print("Execution time:", end_time - start_time, "seconds")
    
# npi_nash.head(3)

**Tasks:**

- First, build a profile of providers referring patients to the major hospitals in Nashville. Are certain specialties more likely to refer to a particular hospital over the others?

In [11]:
top_businesses = npi_nash.groupby("provider_organization_name_(legal_business_name)")["npi"].count()

top_businesses.sort_values(ascending = False).head(20)

provider_organization_name_(legal_business_name)
WALGREEN CO                                 97
RESULTS-AST JV, LLC                         86
KROGER LIMITED PARTNERSHIP I                56
ADVANCED DIAGNOSTIC IMAGING, PC             53
VANDERBILT UNIVERSITY MEDICAL CENTER        39
WAL-MART STORES EAST LP                     37
TENNESSEE ONCOLOGY PHARMACY DISPENSING      35
STATE OF TENNESSEE                          34
PUBLIX TENNESSEE LLC                        30
WAL-MART STORES EAST, LP                    29
TENNESSEE ONCOLOGY PLLC                     28
TENNESSEE CVS PHARMACY, L.L.C.              27
TENNESSEE ORTHOPAEDIC ALLIANCE PA           25
HCA HEALTH SERVICES OF TENNESSEE, INC.      23
MONOGRAM HEALTH PROFESSIONAL SERVICES PC    22
VANDERBILT UNIVERSITY                       20
TENNESSEE DENTAL PROFESSIONALS PC           18
TENNESSEE CVS PHARMACY LLC                  18
CENTERSTONE                                 17
MINDCARE SOLUTIONS, P.C.                    17
Name: npi, 

In [12]:
top_providers = npi_nash.groupby("provider_fullname")["npi"].count()

top_providers.sort_values(ascending = False).head(20)

provider_fullname
SARAH SMITH         7
JESSICA WILLIAMS    7
MICHAEL SMITH       7
LINDA JOHNSON       6
WILLIAM SMITH       6
JENNIFER SMITH      6
JESSICA SMITH       6
ASHLEY JOHNSON      5
JENNIFER JOHNSON    5
JENNIFER ALLEN      5
MARY JOHNSON        5
AMANDA SMITH        5
ASHLEY SMITH        5
SARAH WILLIAMS      5
JESSICA JONES       5
WILLIAM DAVIS       5
JENNIFER GREEN      5
ELIZABETH SMITH     5
MATTHEW SMITH       5
DAVID JOHNSON       5
Name: npi, dtype: int64

In [13]:
query = """
SELECT *
FROM npi 
LEFT JOIN taxonomy
ON npi.healthcare_provider_taxonomy_code = taxonomy.code
INNER JOIN zip_cbsa
ON npi.zip = zip_cbsa.zip
INNER JOIN hopteaming
ON npi.npi = hopteaming.from_npi
WHERE cbsa = 34980;
"""
start_time = time.time()

with sqlite3.connect('../data/hop_teaming.sqlite') as db:
    nashnpi_from_npi  = pd.read_sql(query, db)
    
end_time = time.time()
print("Execution time:", end_time - start_time, "seconds")
    
nashnpi_from_npi.head(3)

Execution time: 34.38408970832825 seconds


,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,...,res_ratio,bus_ratio,oth_ratio,tot_ratio,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1013012616,2.0,"CRIPPS, HOOPER & RHODY, PLLC",None,None,None,None,None,None,400 E PUBLIC SQUARE,...,0.219592,0.043478,0.0,0.211719,1013012616,1093741464,57,72,46.208,58.027
1,1013012616,2.0,"CRIPPS, HOOPER & RHODY, PLLC",None,None,None,None,None,None,400 E PUBLIC SQUARE,...,0.219592,0.043478,0.0,0.211719,1013012616,1104202761,47,65,39.769,38.317
2,1013012616,2.0,"CRIPPS, HOOPER & RHODY, PLLC",None,None,None,None,None,None,400 E PUBLIC SQUARE,...,0.219592,0.043478,0.0,0.211719,1013012616,1295726032,160,303,7.868,28.727


In [14]:
query = """
SELECT *
FROM npi 
LEFT JOIN taxonomy
ON npi.healthcare_provider_taxonomy_code = taxonomy.code
INNER JOIN zip_cbsa
ON npi.zip = zip_cbsa.zip
INNER JOIN hopteaming
ON npi.npi = hopteaming.to_npi
WHERE cbsa = 34980;
"""
start_time = time.time()

with sqlite3.connect('../data/hop_teaming.sqlite') as db:
    nashnpi_to_npi  = pd.read_sql(query, db)
    
end_time = time.time()
print("Execution time:", end_time - start_time, "seconds")
    
nashnpi_to_npi.head(3)

Execution time: 18.708964109420776 seconds


,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,...,res_ratio,bus_ratio,oth_ratio,tot_ratio,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1013012616,2.0,"CRIPPS, HOOPER & RHODY, PLLC",None,None,None,None,None,None,400 E PUBLIC SQUARE,...,0.219592,0.043478,0.0,0.211719,1821080961,1013012616,91,238,9.924,40.266
1,1013012616,2.0,"CRIPPS, HOOPER & RHODY, PLLC",None,None,None,None,None,None,400 E PUBLIC SQUARE,...,0.219592,0.043478,0.0,0.211719,1093741464,1013012616,47,60,36.300,43.394
2,1013012616,2.0,"CRIPPS, HOOPER & RHODY, PLLC",None,None,None,None,None,None,400 E PUBLIC SQUARE,...,0.219592,0.043478,0.0,0.211719,1295726032,1013012616,164,304,7.395,27.308


How many columns does each dataframe have?

In [15]:
print("nashnpi_to_npi has", len(nashnpi_to_npi), "rows")
print("nashnpi_from_npi has", len(nashnpi_from_npi), "rows")

nashnpi_to_npi has 241712 rows
nashnpi_from_npi has 242490 rows


Add a full name provider column to the datasets: 

In [24]:
nashnpi_to_npi["provider_fullname"] = nashnpi_to_npi["provider_first_name"] + " " + nashnpi_to_npi["provider_last_name_(legal_name)"]
nashnpi_from_npi["provider_fullname"] = nashnpi_from_npi["provider_first_name"] + " " + nashnpi_from_npi["provider_last_name_(legal_name)"]


Which classifications/ specializations refer the most? 

In [26]:
top_classifications = nashnpi_from_npi.groupby("classification")["npi"].count()

top_classifications.sort_values(ascending = False).head(15)

classification
Internal Medicine                               61445
Radiology                                       34482
Clinical Medical Laboratory                     14596
Pathology                                       14502
General Acute Care Hospital                     13796
Nurse Practitioner                              12526
Family Medicine                                 10469
Emergency Medicine                               9689
Durable Medical Equipment & Medical Supplies     7013
Anesthesiology                                   6990
Specialist                                       6072
Orthopaedic Surgery                              4478
Nurse Anesthetist, Certified Registered          3905
Clinic/Center                                    3796
Ambulance                                        3753
Name: npi, dtype: int64

Similarly, which classifications/ specializations get the most referral?

In [27]:
most_referred_specialties = nashnpi_to_npi.groupby("classification")["npi"].count()

most_referred_specialties.sort_values(ascending = False).head(15)

classification
Internal Medicine                               65030
Radiology                                       29098
Pathology                                       13857
General Acute Care Hospital                     13755
Nurse Practitioner                              13525
Clinical Medical Laboratory                     13207
Family Medicine                                 10542
Anesthesiology                                   7774
Durable Medical Equipment & Medical Supplies     7649
Emergency Medicine                               6821
Specialist                                       6055
Orthopaedic Surgery                              4842
Nurse Anesthetist, Certified Registered          4275
Psychiatry & Neurology                           4039
Clinic/Center                                    3929
Name: npi, dtype: int64

Which hospitals get the most referrals? 

In [28]:
top_get_referral_hospitals = nashnpi_to_npi.groupby("provider_organization_name_(legal_business_name)")["npi"].count().reset_index(name = "count")

top_get_referral_hospitals.sort_values(by = "count", ascending = False).head(20)

,provider_organization_name_(legal_business_name),count
1266,VANDERBILT UNIVERSITY MEDICAL CENTER,9337
1172,TENNESSEE ONCOLOGY PLLC,5559
89,"ASSOCIATED PATHOLOGISTS, LLC",5529
892,"PATHGROUP LABS, LLC",4180
1017,SAINT THOMAS MEDICAL PARTNERS,3662
15,"ADVANCED DIAGNOSTIC IMAGING, PC",3637
88,ASSOCIATED PATHOLOGISTS LLC,3613
1270,"VERUS HEALTHCARE, LLC",3325
25,AEGIS SCIENCES CORPORATION,2957
960,RADIOLOGY ALLIANCE PC,2835


Which hospitals refer the most?

In [29]:
top_give_referral_hospitals = nashnpi_from_npi.groupby("provider_organization_name_(legal_business_name)")["npi"].count().reset_index(name = "count")

top_give_referral_hospitals.sort_values(by = "count", ascending = False).head(20)

,provider_organization_name_(legal_business_name),count
1265,VANDERBILT UNIVERSITY MEDICAL CENTER,8759
89,"ASSOCIATED PATHOLOGISTS, LLC",5852
1171,TENNESSEE ONCOLOGY PLLC,5562
892,"PATHGROUP LABS, LLC",4981
88,ASSOCIATED PATHOLOGISTS LLC,3893
15,"ADVANCED DIAGNOSTIC IMAGING, PC",3724
1017,SAINT THOMAS MEDICAL PARTNERS,3504
960,RADIOLOGY ALLIANCE PC,3288
25,AEGIS SCIENCES CORPORATION,3203
1269,"VERUS HEALTHCARE, LLC",3101


Make a list of hospitals from the to_npi df and from_npi df. This represents the list of hospitals that receive and give the most referrals. 

In [48]:
# Create a list from the two DataFrames
hospitals_get = top_get_referral_hospitals["provider_organization_name_(legal_business_name)"].tolist()
hospitals_give = top_give_referral_hospitals["provider_organization_name_(legal_business_name)"].tolist()

# Combine the two lists and keep unique elements 
hospitals_list = list(set(hospitals_get + hospitals_give))
# hospitals_list.index
hospitals_list

['PATHOLOGISTS LABORATORY INC',
 'PREMIER EYE GROUP, INC.',
 'FRIENDSHIP HOME HEALTH INC.',
 'TROUSDALE MEDICAL CENTER LLC',
 'SCOTT CRAIG MD',
 'FAMILY PODIATRY CENTER, P.C.',
 'RAI CARE CENTERS OF GALLATIN I LLC',
 'RISHI K SAXENA MD PC',
 'DAVID ALFREDSON MD PC',
 'AHP HOME CARE ALLIANCE OF TENNESSEE',
 'VOLUNTEER BEHAVIORAL HEALTH CARE SYSTEM',
 'CANNON COUNTY HEALTHCARE PC',
 'ONE HUNDRED OAKS IMAGING,LLC',
 'FAMILY CARE CLINIC LLC',
 'CARTER CHIROPRACTIC OFFICES LLC',
 'SAINT THOMAS DEKALB HOSPITAL, LLC',
 'XUHAN PC LLP',
 'DVA RENAL HEALTHCARE INC',
 'EMERGENCY PHYSICIANS AT SUMNER PLLC',
 'LYNN & YORK, OPTOMETRISTS, PC',
 'DAVID A GILPIN MD PLC',
 'RELIANCE MEDICAL ASSOCIATES PLLC',
 'NASHVILLE PAIN & WELLNESS CENTER PLLC',
 'SPRING HILL DERMATOLOGY PLC',
 'APP OF TENNESSEE HM, PLLC',
 'NEBBEN PHYSICAL MEDICINE, LLC',
 'FOOT AND ANKLE HEALTHCARE CENTER',
 'INTERNAL MEDICINE ASSOCIATES OF SOUTHERN HILLS LLC',
 'ASSOCIATED UROLOGISTS OF NASHVILLE',
 'CENTENNIAL NEUROSCIENCE, LLC'

In [49]:
hospitals_list.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

Make a list of private practice providers. 

In [55]:
top_give_referral_providers = nashnpi_from_npi.groupby("provider_fullname")["npi"].count().reset_index(name = "count")

top_give_referral_providers.sort_values(by = "count", ascending = False).head(20)

,provider_fullname,count
1867,DEREK WELCH,2814
3481,JOHN RIDDICK,420
3551,JONATHAN GORDON,409
3908,KATHRYN KNIGHT,396
2974,JASON SHIPMAN,349
2463,GREGORY LASSITER,344
6210,RONALD WATERS,315
722,BENJAMIN GRIFFIN,309
7352,VIRAJ PARIKH,307
5787,PRADEEP JACOB,294


In [56]:
top_get_referral_providers = nashnpi_to_npi.groupby("provider_fullname")["npi"].count().reset_index(name = "count")

top_get_referral_providers.sort_values(by = "count", ascending = False).head(20)

,provider_fullname,count
1865,DEREK WELCH,2330
3479,JOHN RIDDICK,371
3907,KATHRYN KNIGHT,340
7350,VIRAJ PARIKH,284
3549,JONATHAN GORDON,274
36,ADAM GUTTENTAG,273
1919,DONALD VOLLMER,264
868,BRETT MURDOCK,259
721,BENJAMIN GRIFFIN,243
2460,GREGORY LASSITER,239


Who is DEREK WELCH?

In [66]:
subset = nashnpi_to_npi[nashnpi_to_npi["provider_fullname"] == "DEREK WELCH"]
subset.columns
select_columns = [
    'npi', 
    'entity_type_code', 
    'provider_fullname',
    'provider_business_practice_location_address_city_name',
    'code', 'grouping',
    'classification', 'specialization', 'definition', 'notes',
    'display_name', 'section', 'zip', 'cbsa', 'usps_zip_pref_city',
    'usps_zip_pref_state', 'res_ratio', 'bus_ratio', 'oth_ratio',
    'tot_ratio', 'from_npi', 'to_npi', 'patient_count', 'transaction_count',
    'average_day_wait', 'std_day_wait', 'provider_fullname'
]
subset[select_columns].head(3)

,npi,entity_type_code,provider_fullname,provider_business_practice_location_address_city_name,code,grouping,classification,specialization,definition,notes,...,bus_ratio,oth_ratio,tot_ratio,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,provider_fullname
192349,1093753303,1.0,DEREK WELCH,NASHVILLE,207ZP0102X,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,A pathologist deals with the causes and nature...,"Source: American Board of Medical Specialties,...",...,1.0,1.0,1.0,1518302983,1093753303,153,361,22.864,36.255,DEREK WELCH
192350,1093753303,1.0,DEREK WELCH,NASHVILLE,207ZP0102X,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,A pathologist deals with the causes and nature...,"Source: American Board of Medical Specialties,...",...,1.0,1.0,1.0,1518259597,1093753303,34,52,41.173,50.687,DEREK WELCH
192351,1093753303,1.0,DEREK WELCH,NASHVILLE,207ZP0102X,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,A pathologist deals with the causes and nature...,"Source: American Board of Medical Specialties,...",...,1.0,1.0,1.0,1518250794,1093753303,37,67,19.164,31.406,DEREK WELCH


In [68]:

# Create a list from the two DataFrames
providers_get = top_get_referral_providers["provider_fullname"].tolist()
providers_give = top_give_referral_providers["provider_fullname"].tolist()

# Combine the two lists and keep unique elements 
providers_list = list(set(hospitals_get + hospitals_give))

providers_list

['PATHOLOGISTS LABORATORY INC',
 'PREMIER EYE GROUP, INC.',
 'FRIENDSHIP HOME HEALTH INC.',
 'TROUSDALE MEDICAL CENTER LLC',
 'SCOTT CRAIG MD',
 'FAMILY PODIATRY CENTER, P.C.',
 'RAI CARE CENTERS OF GALLATIN I LLC',
 'RISHI K SAXENA MD PC',
 'DAVID ALFREDSON MD PC',
 'AHP HOME CARE ALLIANCE OF TENNESSEE',
 'VOLUNTEER BEHAVIORAL HEALTH CARE SYSTEM',
 'CANNON COUNTY HEALTHCARE PC',
 'ONE HUNDRED OAKS IMAGING,LLC',
 'FAMILY CARE CLINIC LLC',
 'CARTER CHIROPRACTIC OFFICES LLC',
 'SAINT THOMAS DEKALB HOSPITAL, LLC',
 'XUHAN PC LLP',
 'DVA RENAL HEALTHCARE INC',
 'EMERGENCY PHYSICIANS AT SUMNER PLLC',
 'LYNN & YORK, OPTOMETRISTS, PC',
 'DAVID A GILPIN MD PLC',
 'RELIANCE MEDICAL ASSOCIATES PLLC',
 'NASHVILLE PAIN & WELLNESS CENTER PLLC',
 'SPRING HILL DERMATOLOGY PLC',
 'APP OF TENNESSEE HM, PLLC',
 'NEBBEN PHYSICAL MEDICINE, LLC',
 'FOOT AND ANKLE HEALTHCARE CENTER',
 'INTERNAL MEDICINE ASSOCIATES OF SOUTHERN HILLS LLC',
 'ASSOCIATED UROLOGISTS OF NASHVILLE',
 'CENTENNIAL NEUROSCIENCE, LLC'

In [32]:
# This code groups the nashnpi_from_npi by classification then by legal business name 
referral_patterns = nashnpi_from_npi.groupby(["classification", "provider_organization_name_(legal_business_name)"]).size().reset_index(name = "referral_count")

referral_patterns.sort_values(by = "referral_count", ascending = False).head(20)

,classification,provider_organization_name_(legal_business_name),referral_count
970,Pathology,"ASSOCIATED PATHOLOGISTS, LLC",5852
724,Internal Medicine,TENNESSEE ONCOLOGY PLLC,5541
352,Clinical Medical Laboratory,"PATHGROUP LABS, LLC",4981
737,Internal Medicine,VANDERBILT UNIVERSITY MEDICAL CENTER,4709
340,Clinical Medical Laboratory,ASSOCIATED PATHOLOGISTS LLC,3893
563,General Acute Care Hospital,VANDERBILT UNIVERSITY MEDICAL CENTER,3863
626,Internal Medicine,"ADVANCED DIAGNOSTIC IMAGING, PC",3713
709,Internal Medicine,SAINT THOMAS MEDICAL PARTNERS,3473
1158,Radiology,RADIOLOGY ALLIANCE PC,3288
338,Clinical Medical Laboratory,AEGIS SCIENCES CORPORATION,3203
